Homepage: https://rrc.cvc.uab.es/?ch=13&com=downloads. 

In [1]:
!gdown --id 10r9y17wg8Elo-3Zi61xA_8QDaKix8giN -O data.tar.xz

Downloading...
From: https://drive.google.com/uc?id=10r9y17wg8Elo-3Zi61xA_8QDaKix8giN
To: /content/data.tar.xz
247MB [00:01, 139MB/s]


In [2]:
!tar -xf data.tar.xz

In [3]:
import cv2
import glob
import json
import imutils
import numpy as np

# https://stackoverflow.com/a/51855662
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import matplotlib.pyplot as plt

from tqdm import tqdm
from imutils import paths
from google.colab.patches import cv2_imshow

In [4]:
images = sorted(list(paths.list_images("/content/data/img")))
len(images)

626

In [5]:
csvs = sorted(glob.glob("/content/data/box/*.csv"))
len(csvs)

626

In [6]:
jsons = sorted(glob.glob("/content/data/key/*.json"))
len(jsons)

626

In [7]:
csvs[0]

'/content/data/box/000.csv'

In [14]:
sample_csv = pd.read_csv(csvs[0], names=["x1_1", "y1_1", "x2_1", "y2_1", 
                                         "x3_1", "y3_1", "x4_1", "y4_1", "transcript"])
sample_csv.head()

,x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1,transcript
0,72,25,326,25,326,64,72,64,TAN WOON YANN
1,50,82,440,82,440,121,50,121,BOOK TA .K(TAMAN DAYA) SDN BND
2,205,121,285,121,285,139,205,139,789417-W
3,110,144,383,144,383,163,110,163,NO.53 55
4,192,169,299,169,299,187,192,187,TAMAN DAYA


In [9]:
!pip install -q dateparser
import dateparser

     |████████████████████████████████| 286kB 4.1MB/s 


In [15]:
def parse_dates(row):
    return dateparser.parse(row, settings={'STRICT_PARSING': True, 
                                           'REQUIRE_PARTS': ['day', 'month', 'year']})

sample_csv["date_candidate"] = sample_csv["transcript"].apply(parse_dates)
sample_csv["date_candidate"] = pd.to_datetime(sample_csv['date_candidate'], errors = 'coerce').dt.date
sample_csv.sample(5)

,x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1,transcript,date_candidate
39,142,883,353,883,353,901,142,901,***,NaT
42,163,962,330,962,330,977,163,977,PLEASE COME AGAIN !,NaT
19,221,531,247,531,247,545,221,545,RM,NaT
25,202,597,245,597,245,612,202,612,9.000,NaT
17,374,507,441,507,441,521,374,521,AMOUNT,NaT


In [16]:
sample_csv["total_candidate"] = pd.to_numeric(sample_csv["transcript"], errors="coerce")
sample_csv.sample(5)

,x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1,transcript,date_candidate,total_candidate
42,163,962,330,962,330,977,163,977,PLEASE COME AGAIN !,NaT,nan
2,205,121,285,121,285,139,205,139,789417-W,NaT,nan
30,408,669,443,669,443,684,408,684,0.00,NaT,0.000
43,412,639,442,639,442,654,412,654,9.00,NaT,9.000
11,164,397,215,397,215,413,164,413,MANIS,NaT,nan


In [17]:
temp = np.array([False]*sample_csv.shape[0])
for idx, row in sample_csv.iterrows():
  text = str(row['transcript']).replace(':', '').lower()
  if text in ['total', 'amount', 'balance']:
    temp[idx-2:idx+10] = True
sample_csv['total_check'] = temp

In [21]:
sample_csv_filtered = sample_csv[(~sample_csv['date_candidate'].isnull()) | (~sample_csv['total_candidate'].isnull() & sample_csv['total_check'])]
sample_csv_filtered

,x1_1,y1_1,x2_1,y2_1,x3_1,y3_1,x4_1,y4_1,transcript,date_candidate,total_candidate,total_check
9,165,372,342,372,342,389,165,389,25/12/2018 8:13:39 PM,2018-12-25,nan,False
21,27,570,137,570,137,583,27,583,9556939040116,NaT,9556939040116.000,True
25,202,597,245,597,245,612,202,612,9.000,NaT,9.000,True
26,275,598,309,598,309,612,275,612,0.00,NaT,0.000,True
27,411,596,443,596,443,613,411,613,9.00,NaT,9.000,True
30,408,669,443,669,443,684,408,684,0.00,NaT,0.000,True
32,401,703,443,703,443,719,401,719,9.00,NaT,9.000,True
34,402,748,441,748,441,763,402,763,10.00,NaT,10.000,True
36,412,772,443,772,443,786,412,786,1.00,NaT,1.000,True
